In [40]:
filename = "PeerRead/dat/PeerRead/proc/arxiv-all.tf_record.csv"
# read the above csv file in pandas 
import pandas as pd
df = pd.read_csv(filename)

In [41]:
import numpy as np

In [42]:
import numpy as np
from numpy.random import normal, uniform

def make_buzzy_based_simulated_labeler(treat_strength, con_strength, noise_level, setting="simple", seed=0):
    # hardcode probability of theorem given buzzy / not_buzzy
    theorem_given_buzzy_probs = np.array([0.27, 0.07], dtype=np.float32)

    np.random.seed(seed)
    all_noise = np.array(normal(0, 1, 12000), dtype=np.float32)
    all_threshholds = np.array(uniform(0, 1, 12000), dtype=np.float32)

    def labeler(data):
        buzzy = data['buzzy_title']
        index = data['index']
        treatment = data['theorem_referenced']
        treatment = np.float32(treatment)
        confounding = 3.0 * (theorem_given_buzzy_probs[buzzy] - 0.25)

        noise = all_noise[index]

        y, y0, y1 = _outcome_sim(treat_strength, con_strength, noise_level, treatment, confounding, noise,
                                 setting=setting)
        simulated_prob = 1 / (1 + np.exp(-y))
        y0 = 1 / (1 + np.exp(-y0))
        y1 = 1 / (1 + np.exp(-y1))
        threshold = all_threshholds[index]
        simulated_outcome = np.int32(simulated_prob > threshold)

        return {**data, 'outcome': simulated_outcome, 'y0': y0, 'y1': y1, 'treatment': treatment}

    return labeler


In [43]:
def _outcome_sim(beta0, beta1, gamma, treatment, confounding, noise, setting="simple"):
    if setting == "simple":
        y0 = beta1 * confounding
        y1 = beta0 + y0

        simulated_score = (1. - treatment) * y0 + treatment * y1 + gamma * noise

    elif setting == "multiplicative":
        y0 = beta1 * confounding
        y1 = beta0 * y0

        simulated_score = (1. - treatment) * y0 + treatment * y1 + gamma * noise

    elif setting == "interaction":
        # required to distinguish ATT and ATE
        y0 = beta1 * confounding
        y1 = y0 + beta0 * tf.math.square(confounding)

        simulated_score = (1. - treatment) * y0 + treatment * y1 + gamma * noise

    else:
        raise Exception('setting argument to make_simulated_labeler not recognized')

    return simulated_score, y0, y1

In [44]:
import numpy.random as random


In [45]:
def make_extra_feature_cleaning(df):
    # cleaning of extra features from csv file
    df['num_authors'] = np.minimum(df['num_authors'], 6) - 1
    df['year'] = df['year'] - 2007

    # some extras
    equation_referenced = np.minimum(df['num_ref_to_equations'], 1)
    theorem_referenced = np.minimum(df['num_ref_to_theorems'], 1)

    # buzzy title
    any_buzz = df["title_contains_deep"] + df["title_contains_neural"] + \
                df["title_contains_embedding"] + df["title_contains_gan"]
    buzzy_title = np.not_equal(any_buzz, 0).astype(int)

    buzzy_title = np.not_equal(any_buzz, 0).astype(int)
    # add equation_referenced, theorem_referenced, buzzy_title to df
    df['equation_referenced'] = equation_referenced
    df['theorem_referenced'] = theorem_referenced
    df['buzzy_title'] = buzzy_title
    df['index'] = df['id']
    return df

In [46]:
df_new = make_extra_feature_cleaning(df)

/cvmfs/ai.mila.quebec/apps/x86_64/debian/anaconda/3/lib/python3.7/site-packages/pandas/core/computation/expressions.py:194: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  op=op_str, alt_op=unsupported[op_str]


In [47]:
labeler = make_buzzy_based_simulated_labeler(0.5, 5.0, 0.0, 'simple', seed=0)                                          

In [48]:
dick = labeler(df_new)

In [49]:
dick["treatment"] = dick["treatment"].astype(int)

In [50]:
def dick_to_csv(dict, file_name):
    df = pd.DataFrame.from_dict(dict)
    df.to_csv(file_name, index=False)


In [51]:
dick_to_csv(dick, "try.csv")

In [61]:
dick['treatment'].mean()

0.23398433814510802

In [62]:
-0.255206425790686/0.23398433814510802

-1.0906987528046292

In [53]:
reduced_df = pd.read_csv("try.csv")

In [54]:
gt = reduced_df[reduced_df.treatment == 1].y1.mean() - reduced_df[reduced_df.treatment == 1].y0.mean()


In [56]:
naive = reduced_df[reduced_df.treatment == 1].outcome.mean() - reduced_df[reduced_df.treatment == 0].outcome.mean()

In [57]:
naive

0.18506470286742766

In [58]:
gt

0.11149892158470887

In [66]:
dick_to_csv(df_dick, "temp.csv")

In [57]:
df_dick = pd.read_csv("temp.csv")

In [39]:
dick = pd.read_csv('try.csv')

EmptyDataError: No columns to parse from file

In [63]:
df_dick['text'] = df_dick['title'] + "[SEP]" + df_dick['abstract']
df_dick["treatment"] = df_dick["treatment"].astype(int)
df_dick["outcome"] = df_dick["outcome"].apply(lambda x: 1 if x > 0 else 0)

In [36]:
gt = dick[dick.treatment == 1].y1.mean() - dick[dick.treatment == 1].y0.mean()

AttributeError: 'NoneType' object has no attribute 'treatment'

In [51]:
df_dick["outcome"] = df_dick["outcome"].apply(lambda x: 1 if x > 0 else 0)

In [54]:
df_dick["outcome"].sum()


2480

In [55]:
len(df_dick)

10599